### Modelo SIS MP


#### Bibliotecas

In [1]:
import math
import json
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed


#### Busca a paginação

In [2]:
# Configuração do WebDriver
options = webdriver.FirefoxOptions()
options.add_argument('--headless')  # Executa em modo headless (sem interface gráfica)
driver = webdriver.Firefox(options=options)

try:
    # URL
    url = "https://sis.mpsp.mp.br/consultaprocessual/inqueritos-policiais/"
    driver.get(url)

    wait = WebDriverWait(driver, 10)
    paginacao = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "v-data-table-footer__info")))

    # extrai o texto 
    texto_paginacao = paginacao.text
    # converte texto para número
    # dividi o valor por 100 para obter o número de páginas
    # a paginação no link é de 100 em 100
    texto_paginacao = int(texto_paginacao.split("(")[1].split(" ")[0])/100 
    # arredondar para cima
    texto_paginacao = math.ceil(texto_paginacao)
    # print
    print("Texto encontrado:", texto_paginacao)

except Exception as e:
    print(f"Erro ao buscar texto: {e}")
finally:
    # fechar
    driver.quit()

Texto encontrado: 38067


#### Raspa os dados da consulta processual

[Pagina MP](https://sis.mpsp.mp.br/consultaprocessual/)

filtro: inquéritos policiais

Área: criminal

#### Função busca termo "tráfico de drogas e condutas afins" na base

In [3]:
def check_trafico_de_drogas(df):
    # lista
    resultados = []
    
    # iterador
    for x in df['assuntos']:
        # busca o termo 'tráfico de drogas e condutas afins'
        if re.search(r'tráfico de drogas e condutas afins|crimes de tráfico ilícito e uso indevido de drogas', x, re.IGNORECASE):
            resultados.append(True)
        else:
            resultados.append(False)
    
    # Retorna resultados
    return resultados

#### Função para buscar termo "posse de drogas para consumo"

In [4]:
def check_consumo_de_drogas(df):
    # lista
    consumo = []
    
    # iterador
    for x in df['assuntos']:
        # busca o termo 'tráfico de drogas e condutas afins'
        if re.search(r'posse de drogas para consumo pessoal|oferecimento de drogas para consumo conjunto|auxílio ao uso de drogas', x, re.IGNORECASE):
            consumo.append(True)
        else:
            consumo.append(False)
    
    # Retorna resultados
    return consumo

#### Raspador

In [ ]:
def scrape_page(pagina):

    # webdriver  
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    try:
        # URL
        url = f"https://wapiext.mpsp.mp.br/consultaprocessualapispa/consultas/procedimentos-por-classe/?idClasse=705&CurrentPage={pagina}&PageSize=100"
        
        # Acessa a página
        driver.get(url)
        
        # Aguarda um tempo para garantir que a página foi carregada
        time.sleep(0.2)
        
        # Código fonte da página
        page_content = driver.page_source

        # Extrair o JSON usando regex
        json_data_match = re.search(r'(\{.*\})', page_content, re.IGNORECASE)
        if json_data_match:
            json_data = json_data_match.group(1)
            dados_json = json.loads(json_data)
        else:
            raise ValueError("JSON data not found in the page source")

        # Dados extraídos
        dados = []

        for item in dados_json['procedimentos']:
            try:
                page = pagina

                try:
                    idProcedimento = item.get("idProcedimento", None)
                except KeyError:
                    idProcedimento = None
                
                try:
                    numeroMP = item.get("numeroMP", "").lower()
                except KeyError:
                    numeroMP = None
                
                try:
                    anoProcedimento = numeroMP.split("/")[1] if numeroMP else None
                except IndexError:
                    anoProcedimento = None
                
                try:
                    classe = item.get("classe", "").lower()
                except KeyError:
                    classe = None
                
                try:
                    unidadeAdministrativa = item.get("unidadeAdministrativa", "").lower()
                except KeyError:
                    unidadeAdministrativa = None
                
                try:
                    municipio = unidadeAdministrativa.replace('promotoria de justiça de ', '') if unidadeAdministrativa else None
                except AttributeError:
                    municipio = None
                
                try:
                    area = item.get("area", "").lower()
                except KeyError:
                    area = None
                
                try:
                    assuntos = ", ".join([assunto.strip().lower() for assunto in item.get("assuntos", [{}])[0].get("assunto", [])])
                except (KeyError, IndexError):
                    assuntos = None
                
                try:
                    id_link = f"https://wapiext.mpsp.mp.br/consultaprocessualapispa/procedimentos/{idProcedimento}"
                except TypeError:
                    id_link = None
                
                # Adiciona os dados na lista
                dados.append([page, idProcedimento, numeroMP, anoProcedimento, classe, unidadeAdministrativa, municipio, area, assuntos, id_link])

            except Exception as e:
                print(f"Erro ao processar item da página {pagina}: {e}")
                continue  # Continua com o próximo item, caso ocorra um erro em um item específico

        return dados, pagina
    except Exception as e:
        print(f"Tentativa falhou ao buscar a página {pagina}. Erro: {e}")

    finally:
        driver.quit()  # Fecha o driver

# Função principal
def main():
    # variáveis
    dados = []
    url_erro = []

    # ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
        future_to_page = {executor.submit(scrape_page, pagina): pagina for pagina in range(1, texto_paginacao + 1)}
        for future in tqdm(concurrent.futures.as_completed(future_to_page), total=texto_paginacao, desc="Processando páginas"):
            pagina = []
            try:
                dados_extraidos, pagina = future.result()
                dados.extend(dados_extraidos)
            except Exception as e:
                print(f"Erro ao processar a página {pagina} {e}")
                url_erro.append(future_to_page[future])

        # DataFrame
        final = pd.DataFrame(dados, columns=["page","idProcedimento", "numeroMP", "anoProcedimento", "classe", "unidadeAdministrativa", "municipio", "area", "assuntos", "id_link"])

        # coluna 'trafico_validacao'
        final['trafico_validacao'] = check_trafico_de_drogas(final)

        # coluna 'consumo_validacao'
        final['consumo_validacao'] = check_consumo_de_drogas(final)

        # persiste base principal em disco
        final.to_csv("inqueritos_policiais_criminais_mpsp.csv", encoding="cp1252", index=False)

        # persiste erros em disco
        pd.DataFrame(url_erro, columns=["pagina"]).to_csv("inqueritos_policiais_mpsp_url_erro.csv", encoding="cp1252", index=False)

    # erro de URL
    if url_erro:
        print(f"URLs com erro: {url_erro}")

if __name__ == "__main__":
    main() 

Processando páginas:  60%|█████▉    | 22832/38067 [2:45:34<2:51:39,  1.48it/s]

Erro ao processar a página [] Could not reach host. Are you offline?


Processando páginas:  60%|██████    | 22848/38067 [2:45:41<1:34:26,  2.69it/s]

Erro ao processar a página [] Could not reach host. Are you offline?


Processando páginas:  60%|██████    | 22882/38067 [2:45:59<1:37:15,  2.60it/s]

...

Processando páginas:  62%|██████▏   | 23545/38067 [2:52:01<1:36:58,  2.50it/s]

Erro ao processar a página [] Could not reach host. Are you offline?


Processando páginas:  62%|██████▏   | 23555/38067 [2:52:05<1:34:15,  2.57it/s]

Erro ao processar item da página 23530: 'NoneType' object is not subscriptable


Processando páginas:  62%|██████▏   | 23632/38067 [2:52:55<1:45:37,  2.28it/s]


#### Elabora a base que será usada para busca de metados no TJSP

In [ ]:
# importa a base de dados raspada
dataset = pd.read_csv("inqueritos_policiais_criminais_mpsp.csv", encoding="cp1252")

# cria a variavel ano
year = []
for ano in dataset['numeroMP']:
    year.append(ano.split("/")[1])

dataset['anoProcedimento'] = year

# cria dataset sobre tráfico de drogas para raspagem
dataset_trafico = dataset[dataset['trafico_validacao'] == True]

dataset_consumo = dataset[dataset['consumo_validacao'] == True]

dataset_apenas_trafico = dataset[(dataset['consumo_validacao'] == False)&(dataset['trafico_validacao'] == True)]


#### Gráfico do número de processos com inquéritos policiais sobre trafico de drogas

In [ ]:
import plotly.express as px

# Contar os valores de 'trafico_validacao' e resetar o índice
trafico_counts = dataset['trafico_validacao'].value_counts().reset_index()
trafico_counts.columns = ['trafico_validacao', 'count']
trafico_counts['trafico_validacao'] = trafico_counts['trafico_validacao'].replace({True: 'Sim', False: 'Não'})


# Criar o gráfico de barras com Plotly
fig = px.bar(trafico_counts, 
             x='trafico_validacao', y='count', 
             labels={'count': 'Número de casos<br>em milhões', 'trafico_validacao': 'Processos que contém o termo<br>"tráfico de drogas"?'},
             title='Inquéritos Policiais por tráfico e uso de drogas,<br><b>no sistema do Ministério Público do Estado de São Paulo</br></b>',
             text=trafico_counts['count'].apply(lambda x: f'{x:,.0f}'))

# Atualizar layout do gráfico
fig.update_traces(marker_color='cornflowerblue', 
                  textposition='inside')
fig.update_layout(width=600, 
                  height=400, 
                  xaxis_showgrid=False, 
                  yaxis_showgrid=False, 
                  template='plotly_white')

# Exibir o gráfico
fig.show()

In [ ]:
dataset['trafico_validacao'].value_counts().mul(100)

In [ ]:
dataset.shape

#### Gráfico da distribuição por ano dos inquéritos sobre tráfico e consumo de drogas que entraram no MP-SP

In [ ]:
import plotly.express as px

# Contar os valores de 'anoProcedimento' e resetar o índice
ano_counts = dataset_apenas_trafico['anoProcedimento'].value_counts().sort_index().reset_index()
ano_counts.columns = ['anoProcedimento', 'count']
ano_counts = ano_counts[1:]

# Criar o gráfico de barras com Plotly
fig = px.bar(ano_counts, 
             x='anoProcedimento', y='count', 
             labels={'count': 'Número de Inquéritos<br>em milhares', 'anoProcedimento': 'Ano de entrada<br>no sistema do MP-SP'},
             title='<b>Evolução do Inquéritos Policiais registrados por tráfico de drogas,<br>no sistema do Ministério Público do Estado de São Paulo</br></b>',
             text=ano_counts['count'].apply(lambda x: f'{x:.0f}'))

# Atualizar layout do gráfico
fig.update_traces(marker_color='cornflowerblue', 
                  textposition='inside')
fig.update_layout(width=1000, 
                  height=500, 
                  xaxis_showgrid=False, 
                  yaxis_showgrid=False, 
                  template='plotly_white')

# Exibir o gráfico
fig.show()

In [ ]:
# variação percentual entre 2017 e 2023
# [data_atual] - ['data_anterior'] / ['data_anterior'] * 100.
(176261-361568)/361568*100

In [ ]:
uso_counts = dataset_trafico['consumo_validacao'].value_counts().sort_index().reset_index()
uso_counts.columns = ['consumo_validacao', 'count']
uso_counts['consumo_validacao'] = uso_counts['consumo_validacao'].replace({True: 'Consumo', False: 'Tráfico'})

# Criar o gráfico de barras com Plotly
fig = px.bar(uso_counts, 
             x='consumo_validacao', y='count', 
             labels={'count': 'Número de Inquéritos<br>em milhares', 'consumo_validacao': 'Tipo de Inquérito'},
             title='<b>Inquéritos Policiais por tráfico e uso de drogas,<br>no sistema do Ministério Público do Estado de São Paulo</br></b>',
             text=uso_counts['count'].apply(lambda x: f'{x:,.0f}'))

# Atualizar layout do gráfico
fig.update_traces(marker_color='cornflowerblue', 
                  textposition='outside')

fig.update_layout(width=600, 
                  height=500, 
                  xaxis_showgrid=False, 
                  yaxis_showgrid=False, 
                  template='plotly_white')

# Exibir o gráfico
fig.show()

In [ ]:
9644/298842*100

In [ ]:
import plotly.express as px

# Contar os valores de 'consumo_validacao' por 'anoProcedimento'
consumo_ano_counts = dataset_trafico.groupby(['anoProcedimento', 'consumo_validacao']).size().reset_index(name='count')

# Substituir os valores booleanos por strings mais legíveis
consumo_ano_counts['consumo_validacao'] = consumo_ano_counts['consumo_validacao'].replace({True: 'Consumo', False: 'Tráfico'})
consumo_ano_counts = consumo_ano_counts[2:]

# Criar o gráfico de barras empilhadas com Plotly
fig = px.bar(consumo_ano_counts, 
             x='anoProcedimento', y='count', color='consumo_validacao', 
             labels={'count': 'Número de Inquéritos<br>em milhares', 'anoProcedimento': 'Ano de Procedimento', 'consumo_validacao': 'Tipo de Inquérito'},
             title='<b>Distribuição de Inquéritos Policiais por ano e tipo de infração<b>',
             text=consumo_ano_counts['count'].apply(lambda x: f'{x:.0f}'),
             color_discrete_map={'Tráfico': 'cornflowerblue', 'Consumo': 'RGB(113,50,141)'})

# Atualizar layout do gráfico
fig.update_layout(barmode='stack', 
                  xaxis={'categoryorder':'category ascending'},
                  template='plotly_white')
# formata o gráfico
fig.update_layout(xaxis_showgrid=False, 
                  yaxis_showgrid=False, 
                  template='plotly_white')

# Exibir o gráfico
fig.show()


In [ ]:
consumo_ano_counts1 = consumo_ano_counts[consumo_ano_counts['consumo_validacao'] == 'Consumo']
# Criar o gráfico de barras empilhadas com Plotly
fig = px.bar(consumo_ano_counts1, 
             x='anoProcedimento', y='count', color='consumo_validacao', 
             labels={'count': 'Número de Inquéritos', 'anoProcedimento': 'Ano de Procedimento', 'consumo_validacao': 'Tipo de Inquérito'},
             title='<b>Distribuição de Inquéritos Policiais por ano e tipo de infração<b>',
             text=consumo_ano_counts1['count'].apply(lambda x: f'{x:.0f}'),
             color_discrete_map={'Consumo': 'cornflowerblue'})

# Atualizar layout do gráfico
fig.update_layout(barmode='stack', 
                  xaxis={'categoryorder':'category ascending'},
                  template='plotly_white')
# formata o gráfico
fig.update_layout(xaxis_showgrid=False, 
                  yaxis_showgrid=False, 
                  template='plotly_white')

# Exibir o gráfico
fig.show()

In [ ]:
dataset_trafico.head(2)

#### Usa o número do processo para pegar o PIN de cada ação da base

#### Raspa os metadados de cada processo de tráfico

In [ ]:
link = [f"https://wapiext.mpsp.mp.br/consultaprocessualapispa/procedimentos/{x}" for x in dataset_apenas_trafico['idProcedimento']]

In [ ]:
def lista_para_string(lista, chave='assunto'):
    """Transforma listas de dicionários em strings"""
    return ",".join(["".join(item.get(chave, [])).title() for item in lista])

def extrair_dados_processo(link):
    # Configuração do WebDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    try:        
        # Acessa a URL
        driver.get(link)
        
        # Aguardar um pouco para garantir que a página carregue completamente
        time.sleep(0.5)

        # Pega o código fonte da página
        page_content = driver.page_source
        
        # Extrai o JSON do page_source usando regex
        json_data_match = re.search(r'(\{.*\})', page_content)
        if json_data_match:
            json_data = json_data_match.group(1)
            dados_processo = json.loads(json_data)
        else:
            raise ValueError("JSON data not found in the page source")
        
        data = {}

        # Usando try-except para garantir que não falhe caso algum campo não exista

        try:
            data['situacao'] = dados_processo['situacao']
        except KeyError:
            data['situacao'] = None
        
        try:
            data['numeroMp'] = dados_processo['numeroMp']
        except KeyError:
            data['numeroMp'] = None
        
        try:
            data['numerosTJ'] = lista_para_string(dados_processo['numerosTJ'], 'numero')
        except KeyError:
            data['numerosTJ'] = None
        except Exception as e:
            #print(f"Erro ao processar numerosTJ: {e}")
            data['numerosTJ'] = None
        
        try:
            data['classe'] = dados_processo['classe']
        except KeyError:
            data['classe'] = None
        
        try:
            data['unidadeAdministrativa'] = dados_processo['unidadeAdministrativa']
        except KeyError:
            data['unidadeAdministrativa'] = None
        
        try:
            data['cargo'] = dados_processo['cargo']
        except KeyError:
            data['cargo'] = None
        
        try:
            data['dataInstauracao'] = dados_processo['dataInstauracao']
        except KeyError:
            data['dataInstauracao'] = None
        
        try:
            data['numeroInqueritoDelegacia'] = dados_processo['numeroInqueritoDelegacia']
        except KeyError:
            data['numeroInqueritoDelegacia'] = None
        
        try:
            data['nomeDelegacia'] = dados_processo['nomeDelegacia']
        except KeyError:
            data['nomeDelegacia'] = None
        
        try:
            data['assuntos'] = lista_para_string(dados_processo['assuntos'], 'assunto')
        except KeyError:
            data['assuntos'] = None
        except Exception as e:
            #print(f"Erro ao processar assuntos: {e}")
            data['assuntos'] = None
        
        try:
            data['partes1'] = lista_para_string(dados_processo['partes'], 'nome')
        except KeyError:
            data['partes1'] = None
        except Exception as e:
            #print(f"Erro ao processar partes1: {e}")
            data['partes1'] = None
        
        try:
            data['participacao1'] = lista_para_string(dados_processo['partes'], 'participacao')
        except KeyError:
            data['participacao1'] = None
        except Exception as e:
            #print(f"Erro ao processar participacao1: {e}")
            data['participacao1'] = None
        
        try:
            data['ultimoMovimentos'] = lista_para_string(dados_processo['movimentos'], 'nome')
        except KeyError:
            data['ultimoMovimentos'] = None
        except Exception as e:
            #print(f"Erro ao processar ultimoMovimentos: {e}")
            data['ultimoMovimentos'] = None
        
        try:
            data['dataUltimoMovimento'] = lista_para_string(dados_processo['movimentos'], 'data')
        except KeyError:
            data['dataUltimoMovimento'] = None
        except Exception as e:
            #print(f"Erro ao processar dataUltimoMovimento: {e}")
            data['dataUltimoMovimento'] = None
        
        # Tenta adicionar anexos e outras informações, caso existam
        try:
            data['anexos'] = lista_para_string(dados_processo['anexos'], 'tipo')
        except KeyError:
            data['anexos'] = None
        except Exception as e:
            #print(f"Erro ao processar anexos: {e}")
            data['anexos'] = None
        
        try:
            data['anonimo1'] = lista_para_string(dados_processo['partes'], 'anonimo')
        except KeyError:
            data['anonimo1'] = None
        except Exception as e:
            #print(f"Erro ao processar anonimo1: {e}")
            data['anonimo1'] = None
        
        try:
            data['desconhecido1'] = lista_para_string(dados_processo['partes'], 'desconhecido')
        except KeyError:
            data['desconhecido1'] = None
        except Exception as e:
            #print(f"Erro ao processar desconhecido1: {e}")
            data['desconhecido1'] = None
        
        # URL final do processo
        data['url'] = link

        return data

    except Exception as e:
        #print(f"Erro ao extrair dados do processo: {e}")
        return None
    finally:
        driver.quit()  # Certifica-se de fechar o driver depois


#####################
### link raspagem ###
#####################
link = [x for x in dataset_apenas_trafico['id_link']]

def main():
    # Inicializar a lista para armazenar os dados dos processos
    dados_processos = []

    # Usando ThreadPoolExecutor para paralelizar a extração de dados dos processos
    with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
        future_to_link = {executor.submit(extrair_dados_processo, link): link for link in tqdm(dataset_apenas_trafico['id_link'], desc="Processando links")}
        for future in tqdm(concurrent.futures.as_completed(future_to_link), total=len(future_to_link), desc="Extraindo dados"):
            link = future_to_link[future]
            try:
                data = future.result()
                dados_processos.append(data)
            except Exception as e:
                print(f"Erro ao processar o link {link}: {e}")

            # Exibir os dados extraídos
            df = pd.DataFrame(dados_processos)
            df.to_csv("metadados_mpsp_processos_trafico.csv", encoding="cp1252", index=False)

if __name__ == "__main__":
    main()


In [ ]:
pd.read_csv("metadados_mpsp_processos_trafico.csv", encoding="cp1252")

In [ ]:
# # pagina atual
# pagina = 1

# # dados extraídos
# dados = []
# id_link = []
# url_erro = []

# # Inicializar o driver
# driver = webdriver.Chrome()

# # Loop
# for pagina in tqdm(range(1, texto_paginacao + 1)):
#     try:
#         # Acessar URL - Inquéritos Policiais - Criminal
#         url = f"https://wapiext.mpsp.mp.br/consultaprocessualapispa/consultas/procedimentos-por-classe/?idClasse=705&CurrentPage={pagina}&PageSize=100"
#         #print(url)
#         driver.get(url)

#         # time
#         time.sleep(0.5)

#         # pega código fonte da página
#         page_content = driver.page_source

#         # Extrair o JSON do page_source usando regex
#         json_data = re.search(r'(\{.*\})', page_content).group(1)
#         dados_json = json.loads(json_data)
#         #print(dados_json)
        
#         # Formação da base de dados
#         for item in dados_json['procedimentos']:
#             idProcedimento = item["idProcedimento"]
#             numeroMP = item["numeroMP"].lower()
#             classe = item["classe"].lower()
#             unidadeAdministrativa = item["unidadeAdministrativa"].lower()
#             municipio = item["unidadeAdministrativa"].lower().replace('promotoria de justiça de ', '')
#             area = item["area"].lower()
#             assuntos = ", ".join([assunto.strip().lower() for assunto in item["assuntos"][0]["assunto"]])
#             trafico = assuntos.find("tráfico")
#             dados.append([idProcedimento, numeroMP, classe, unidadeAdministrativa, municipio, area, assuntos])
#             id_link.append("https://sis.mpsp.mp.br/consultaprocessual/inqueritos-policiais/procedimento/" + idProcedimento)

#         pagina+=1
        
#     except Exception as e:
#         print(f"Erro ao buscar texto: {e}")
#         # salva url com erro
#         url_erro.append(url)
#         pass

# # Fechar navegador
# driver.quit()

# # cria dataframe
# final = pd.DataFrame(dados, columns=["idProcedimento", "numeroMP", "classe", "unidadeAdministrativa", "municipio", "area", "assuntos"])

# # cria coluna classificação tráfico de drogas
# # Chamando a função
# final['trafico_validacao'] = check_trafico_de_drogas(final)

# # salva csv
# final.to_csv("inqueritos_policiais_criminal.csv", encoding="cp1252", index=False)

In [ ]:
# def extracao_pin_processo_mp(numero):
#     # webDriver
#     driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))  # Garantir que o ChromeDriver seja instalado
#     try:
#         # formata numero processo
#         num_limpo = re.sub(r'[^\w\s]', '', numero)

#         # gera url
#         url = f"https://wapiext.mpsp.mp.br/consultaprocessualapispa/procedimentos/numero-mp/{num_limpo}/id?CenarioDeUso=0"
#         print(url)
#         try:        
#             # acessa url
#             driver.get(url)

#             # Aguardar um pouco para garantir que a página carregue completamente
#             time.sleep(1)

#             # Pega o código fonte da página
#             page_content = driver.page_source
            
#             # Extrai o JSON do page_source usando regex
#             json_data_match = re.search(r'([a-z0-9-]{36})', page_content)
#             if json_data_match:
#                 uuid = json_data_match.group(1)
#             else:
#                 raise ValueError("JSON data not found in the page source")
            
#             # verifica se existe o pin
#             if uuid:
#                 # retorna o link para raspagem json()
#                 return f"https://wapiext.mpsp.mp.br/consultaprocessualapispa/procedimentos/{uuid}"
#             else:
#                 print("PIN não encontrado no conteúdo.")
#                 pass
#         except Exception as e:
#             print(f"Erro ao buscar PIN do processo {numero}: {e}")
#             pass
        
#     except Exception as e:
#         print(f"Erro ao buscar página {url}: {e}")
#         pass

#     finally:
#         # fecha o WebDriver
#         driver.quit()

# # Executa processamento paralelo
# def extracao_pin_processo_mp_parallel(numeros_processos):
#     with ThreadPoolExecutor(max_workers=5) as executor:
#         results = list(tqdm(executor.map(extracao_pin_processo_mp, numeros_processos), total=len(numeros_processos), desc="Processando processos"))
#     return results

# # aplicando as funções paralelas
# numeros_processos = list(dataset_apenas_trafico['numeroMP'][0:20])
# links = extracao_pin_processo_mp_parallel(numeros_processos)

# with open("links_raspagem.json", "w") as f:
#     json.dump(links, f)

# print(links)